### Simple notebook that translates the texts, questions and answers to English

Note:
- Uses Google Translate, which can get quite expensive
- my GCS credentials json file are in a private dataset here

In [ ]:
import pandas as pd
from IPython.display import Markdown

In [ ]:
train_df = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/train.csv")

# Adding Google API translations

In [ ]:
from google.cloud import translate_v2 as translate

In [ ]:
def translate_text(target, text, translate_client):
    try:
        result = translate_client.translate(text, target_language=target)
        translatedText = result["translatedText"]
        return translatedText
    except Exception as e:
        return ''

In [ ]:
credentials_path = "../input/my-google-credentials/sa_credentials.json"
translate_client = translate.Client.from_service_account_json(credentials_path)

for text_column in ["context", "question", "answer_text"]:
    train_df[f"{text_column}_gtrans"] = train_df[text_column].apply(lambda x: translate_text("en", x, translate_client))

In [ ]:
train_df.head()

# Looking at results

## Google Translate

In [ ]:
sample_train_df = train_df.sample(4, random_state=1) # two of hindi and two of tamil

new_line = "\n"
markdown = f"# GOOGLE TRANSLATION  {new_line}  "

for idx, example in sample_train_df.iterrows():
    markdown += f"{example['context_gtrans']}  {new_line}{new_line}  "
    markdown += f"**Question**: {example['question_gtrans']},  "  
    markdown += f"**Answer**: {example['answer_text_gtrans']},  "  
    markdown += f"**Language**: {example['language']}.  {new_line}{new_line}  "
        
Markdown(markdown)

In [ ]:
train_df.to_csv("train_with_google_translations.csv")